In [25]:
# See the constants defined in: https://github.com/zkcrypto/jubjub?tab=readme-ov-file

# the base field of JubJub
p = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001
Fp = GF(p) 
# Montgomery curve: m(x, y) = x^3 + J * x^2 - K * y^2 + x
# Constants copied from: https://github.com/zkcrypto/jubjub
MONT_J = Fp(40962)
MONT_K = Fp(1)

In [26]:
# Weierstrass curve: w(x, y) = x^3 - y^2 + A * x + B
# Transform relation between Montgomery and Weierstrass, see: https://www.rfc-editor.org/rfc/rfc9380.html#appx-rational-map
WEI_A = (Fp(3) - MONT_J ** 2) / (Fp(3) * MONT_K ** 2)
WEI_B = (Fp(2) * MONT_J ** 3 - Fp(9) * MONT_J) / (Fp(27) * MONT_K ** 3)

In [27]:
(WEI_A, WEI_B)

(52435875175126190479447740508185965837690552500527637822603658699938021889366,
 5091077286874)

In [28]:
# Edwards curve: e(x, y) = a * x^2 + y^2 - 1 - d * x^2 * y^2
# Constants copied from: https://github.com/zkcrypto/jubjub
ED_a = Fp(-1)
ED_d = Fp(-1) * Fp(10240) / Fp(10241)

In [29]:
P.<x, y> = PolynomialRing(Fp, 2)
P

Multivariate Polynomial Ring in x, y over Finite Field of size 52435875175126190479447740508185965837690552500527637822603658699938581184513

In [30]:
K = FractionField(P)
K

Fraction Field of Multivariate Polynomial Ring in x, y over Finite Field of size 52435875175126190479447740508185965837690552500527637822603658699938581184513

In [31]:
x, y = K.gens()

In [32]:
montgomery_form = x * x * x + MONT_J * x ** 2 - MONT_K * y * y  + x
montgomery_form

x^3 + 40962*x^2 - y^2 + x

In [33]:
weierstrass_form = x ** 3 - y * y + WEI_A * x + WEI_B
weierstrass_form

x^3 - y^2 + 52435875175126190479447740508185965837690552500527637822603658699938021889366*x + 5091077286874

In [34]:
edwards_form = ED_a * x ** 2 + y ** 2 - 1 - ED_d * x ** 2 * y ** 2
edwards_form

33178837138445241119697427838399087845741117098273517536419461807987697107280*x^2*y^2 - x^2 + y^2 + 52435875175126190479447740508185965837690552500527637822603658699938581184512

In [35]:
# Given (x, y) in Weierstrass curve, convert to the point in Montgomery curve
# Algorithm from: https://www.rfc-editor.org/rfc/rfc9380.html#appendix-D.2-5
def weierstrass_to_montgomery(x, y):
    return ((3 * MONT_K * x - MONT_J) / 3, y * MONT_K)

montgomery_form(weierstrass_to_montgomery(x, y)) == weierstrass_form

True

In [36]:
# See details here: 
# https://www.ietf.org/archive/id/draft-irtf-cfrg-hash-to-curve-10.html#appx-rational-map

In [37]:
# Algorithm from: https://www.rfc-editor.org/rfc/rfc9380.html#appendix-D.1-12
def montgomery_to_edwards(s, t):
    tv1 = s + 1
    tv2 = tv1 * t        # (s + 1) * t
    # tv2 = inv0(tv2)      # 1 / ((s + 1) * t)
    if tv2 != 0:
        tv2 = 1 / tv2
    v = tv2 * tv1      # 1 / t
    v = v * s          # s / t
    w = tv2 * t        # 1 / (s + 1)
    tv1 = s - 1
    w = w * tv1        # (s - 1) / (s + 1)
    # e = tv2 == 0
    # w = CMOV(w, 1, e)  # handle exceptional case
    if tv2 == 0:
        w = 1
    return (v, w)

In [38]:
# Results as expected, see: https://www.rfc-editor.org/rfc/rfc9380.html#appendix-D.1-9
(u, v) = montgomery_to_edwards(x, y)
(u, v)

(x/y,
 (x + 52435875175126190479447740508185965837690552500527637822603658699938581184512)/(x + 1))

In [39]:
E = EllipticCurve(Fp, [WEI_A, WEI_B])
E

Elliptic Curve defined by y^2 = x^3 + 52435875175126190479447740508185965837690552500527637822603658699938021889366*x + 5091077286874 over Finite Field of size 52435875175126190479447740508185965837690552500527637822603658699938581184513

In [40]:
P = E.random_point()
(a, b) = P.xy()
(a, b)

(32918843519861289097070882940437200891225887917683614207709323528149518806503,
 6871706826388728304137269180931183552457562826747428960849902242926391355564)

In [41]:
weierstrass_form(a, b)

0

In [42]:
(c, d) = weierstrass_to_montgomery(a, b)
montgomery_form(c, d) == 0

True

In [43]:
# This means the point is not on the corresponding Edwards curve!!!
(e, f) = montgomery_to_edwards(c, d)
edwards_form(e, f) == 0

False

In [44]:
# test n random points for the correctness
def test(n):
    for i in range(n):
        E = EllipticCurve(Fp, [WEI_A, WEI_B])
        P = E.random_point()
        (a, b) = P.xy()
        not_on_weierstrass = weierstrass_form(a, b) != 0

        (c, d) = weierstrass_to_montgomery(a, b)
        not_on_montgomery = montgomery_form(c, d) != 0

        (e, f) = montgomery_to_edwards(c, d) 
        not_on_edwards = edwards_form(e, f) != 0
    
        if not_on_montgomery | not_on_weierstrass | not_on_edwards:
            print("point not on the curve!")
            return
    
    print("pass")

test(10)

point not on the curve!


In [45]:
def find_z_svdw(F, A, B, init_ctr=1):
    g = lambda x: F(x)^3 + F(A) * F(x) + F(B)
    h = lambda Z: -(F(3) * Z^2 + F(4) * A) / (F(4) * g(Z))
    # NOTE: if init_ctr=1 fails to find Z, try setting it to F.gen()
    ctr = init_ctr
    while True:
        for Z_cand in (F(ctr), F(-ctr)):
            # Criterion 1:
            #   g(Z) != 0 in F.
            if g(Z_cand) == F(0):
                continue
            # Criterion 2:
            #   -(3 * Z^2 + 4 * A) / (4 * g(Z)) != 0 in F.
            if h(Z_cand) == F(0):
                continue
            # Criterion 3:
            #   -(3 * Z^2 + 4 * A) / (4 * g(Z)) is square in F.
            if not is_square(h(Z_cand)):
                continue
            # Criterion 4:
            #   At least one of g(Z) and g(-Z / 2) is square in F.
            if is_square(g(Z_cand)) or is_square(g(-Z_cand / F(2))):
                return Z_cand
        ctr += 1

In [46]:
jubjub_z = find_z_svdw(Fp, WEI_A, WEI_B)
jubjub_z

2

In [47]:
kronecker(7, p) # i.e 7 is a quadratic non-residue

-1